In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os 
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter("username","password")


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='left-text',
            className='col s12 m6',
             children=[
                 html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
                html.H2("Nicholas Boodoo Project 2 Dashboard"),
             ]
            ),
        html.Div(
            id='right-text',
            className='col s12 m6',
            children=[html.Img(height="100",src='data:image/png;base64,{}'.format(encoded_image.decode()))]
            )
        ]),
    
    
    html.Hr(),
    html.Div(
        dcc.Dropdown(
            id="filter-type",
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain/Wilderness', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'}
            ],
            placeholder="Select a Filter",
            multi=False
        ) 


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

        #Filters: water, mountain, disaster.
    
        #Water filter - Labrador Retriever Mix, Chesapeake Bay Retriever, Newfoundland
        #Intact Female 26 weeks to 156 weeks
        if filter_type == 'water' :
             filter = {"$and": [{
                  "breed": {"$in" :[
                         "Labrador Retriever Mix",
                         "Chesapeake Bay Retriever",
                         "Newfoundland"
                     ]},  

                     "sex_upon_outcome": "Intact Female",
                     "age_upon_outcome_in_weeks":  {'$gt':26, '$lt':156}
                 }]}
            #Mountain filter - German shepherd, alaskan malamute, old english sheepdog,
            # siberian husky, rottweiler.
            # Intact male 26 to 156 weeks
        elif filter_type == 'mountain':
            filter = {"$and": [{
                  "breed": {"$in" :[
                         "German Shepherd",
                         "Alaskan Malamute",
                         "Old English Sheepdog",
                         "Siberian Husky",
                         "Rottweiler"
                     ]},  

                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks":  {'$gt':26, '$lt':156}
                 }]}
            #Disaster filter - Doberman, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
            #Intact male 20-300 weeks
        elif filter_type == "disaster":
            filter = {"$and": [{
                  "breed": {"$in" :[
                         "Doberman Pinscher",
                         "German Shepherd",
                         "Golden Retriever",
                         "Bloodhound",
                         "Rottweiler"
                     ]},  

                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks":  {'$gt':20, '$lt':300}
                 }]}
        else:
            filter = {}
        df = pd.DataFrame.from_records(shelter.read(filter))
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    return [
        dcc.Graph(            
            figure = px.pie(df,names='breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ] 



app